In [1]:
import torch
import torch.functional as F
import torch.nn as nn
import spacy
import pandas as pd
import numpy as np
import random
import os
import time
from tqdm.notebook import tqdm
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from spacy.lang.en.stop_words import STOP_WORDS
SEED=42
torch.manual_seed(SEED)
import re

In [2]:
dataset = pd.read_csv('/home/karthiktiwari/Documents/Projects/sarcasm_detection/Sarcasm_dataset.csv')

In [3]:
len(dataset[dataset['sarcastic']==1]) , len(dataset[dataset['sarcastic']==0])

(47930, 49597)

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
for train_index, test_index in sss.split(dataset.loc[:, 'tweet_id'], dataset.loc[:, 'sarcastic']):
    train_df = dataset.iloc[train_index,:]
    test_df = dataset.iloc[test_index,:]

In [5]:
root_dir = '/home/karthiktiwari/Data/Sarcasm Detection/'

In [6]:
# err_ids = []
# with open('../err_ids.txt') as file:
#     err_ids = file.readlines()
# err_ids = [int(idx) for idx in err_ids]

# err_i = []
# for i in tqdm(range(len(dataset))):
#     if dataset.loc[i,'tweet_id'] in err_ids:
#         err_i.append(i)

# dataset.drop(index = err_i, axis = 0, inplace=True)
# dataset.to_csv('/home/karthiktiwari/Documents/Projects/sarcasm_detection/Sarcasm_dataset.csv', )

In [7]:
class Vocabulary(object):
    """Class to build vocabulary for mapping"""
    def __init__(self, token_to_idx = None, add_unk = True, unk_token = "<UNK>", mask_token="<MASK>", begin_seq_token="<BEGIN>",end_seq_token="<END>"):
        """
        Args:
            token_to_idx: Initialize token to idx dictionary.
            add_unk: Whether to include the unknown token in the vocabulary
            unk_token: How the unknown token is represented in the vocabulary
        """
        
        if token_to_idx is None:
            token_to_idx = {}
            
        self._token_to_idx = token_to_idx
        
        self._idx_to_token = {idx:token 
                             for token, idx in self._token_to_idx.items()}
        
#         self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

#         self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)
        
        self._add_unk = add_unk
        
    def from_serializable(self):
        """
        Returns:
            A dictionary that can be serialized
        """

        return cls(**contents)
        
    def add_token(self, token):
        """ Update mapping dictionaries given the token
        Args:
            token (str): Token to add to the vocabulary
        Returns:
            index (int): The index corresponding to the token          
        """

        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token

        return index
        
    def lookup_token(self, token):
        """ Retrieves the index associated with the token
            or the UNK index if the token isn't present in the vocabulary

        Args:
            token (str): The token for which the index has to be retrieved
        Returns:
            index (int): The index associated with the token in the dictionary

        Note: 
            'UNK Index' has to be >=0 for the UNK functionality
        """

        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx.get(token)

    def lookup_index(self, index):
        """Retrieve the token associated with the index
        Args:
            index (int): The index to look up
        Returns:
            token (str): The token associated with the index
        Raises:
            KeyError: if the index is not in the vocabulary
        """

        if index not in self._idx_to_token:
            raise KeyError("the index %d is not in the vocabulary" % index)
        else:
            return self._idx_to_token[index]

    def __len__(self):
        """Returns the length of the vocabulary
        """
        return len(self._token_to_idx)      


In [8]:
class Vocabulary(object):
    """Class to build vocabulary for mapping"""
    def __init__(self, token_to_idx = None, add_unk = True, unk_token = "<UNK>", mask_token="<MASK>", begin_seq_token="<BEGIN>",end_seq_token="<END>"):
        """
        Args:
            token_to_idx: Initialize token to idx dictionary.
            add_unk: Whether to include the unknown token in the vocabulary
            unk_token: How the unknown token is represented in the vocabulary
        """
        
        if token_to_idx is None:
            token_to_idx = {}
            
        self._token_to_idx = token_to_idx
        
        self._idx_to_token = {idx:token 
                             for token, idx in self._token_to_idx.items()}
        
#         self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

#         self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)
        
        self._add_unk = add_unk
        
    def from_serializable(self):
        """
        Returns:
            A dictionary that can be serialized
        """

        return cls(**contents)
        
    def add_token(self, token):
        """ Update mapping dictionaries given the token
        Args:
            token (str): Token to add to the vocabulary
        Returns:
            index (int): The index corresponding to the token          
        """

        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token

        return index
        
    def lookup_token(self, token):
        """ Retrieves the index associated with the token
            or the UNK index if the token isn't present in the vocabulary

        Args:
            token (str): The token for which the index has to be retrieved
        Returns:
            index (int): The index associated with the token in the dictionary

        Note: 
            'UNK Index' has to be >=0 for the UNK functionality
        """

        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx.get(token)

    def lookup_index(self, index):
        """Retrieve the token associated with the index
        Args:
            index (int): The index to look up
        Returns:
            token (str): The token associated with the index
        Raises:
            KeyError: if the index is not in the vocabulary
        """

        if index not in self._idx_to_token:
            raise KeyError("the index %d is not in the vocabulary" % index)
        else:
            return self._idx_to_token[index]

    def __len__(self):
        """Returns the length of the vocabulary
        """
        return len(self._token_to_idx)      


In [9]:
class Sarcastic_tweets(Dataset):
    def __init__(self, df, nlp, vocab):
        """Initializing
        Args:
            df (Pandas DataFrame): Dataframe consisting of tweets and labels
            nlp (spacy object): For preprocessing
            vocab (Vocabulary Object): To vectorize the tweets
        """
        self.df = df
        self.nlp = nlp
        self.vocab = vocab
        measure_len = lambda context: len(context.split(" "))
        # self._max_seq_length = max(map(measure_len, self.df.tweet_id)) + 1
        
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        with open(os.path.join(root_dir, str(self.df.tweet_id.iloc[idx])+'.txt')) as file:
              tweet = file.read()
        
        label = self.df.sarcastic.iloc[idx]
        return {'tweet':torch.LongTensor(self.preprocess(tweet)), 'label':label}
        # return {'tweet':self.preprocess(tweet), 'label':label}
    
    def preprocess(self, sent):
        
        #Preprocessing and tokenizing
        sent = re.sub(r'RT','',sent)
        sent = re.sub('@[a-zA-z0-9_]+','',sent)
        sent = re.sub(r'http://[^\s<>"]+|www\.[^\s<>"]+','',sent)
        sent = re.sub(r'&#[0-9]+','',sent)
        sent = re.sub(r'[0-9]+','',sent)
        sent = re.sub(r'[^\w\s]+', '',sent)
        sent = sent.strip()
        sent = re.sub('[^a-zA-Z]',' ',sent)
        sent = " ".join(sent.split())
        sent = [token.lemma_ for token in nlp(sent) if token.text not in STOP_WORDS]
        sent = self.vectorize(sent)
        return sent

    def vectorize(self, sent):
        """Converts raw text to numeric vectors using the vocabulary
        Args:
            sent (str): The tweet to be vectorized
        Returns:
            vector (list): The vector associated with the tweet
        """
        vector = [self.vocab.begin_seq_index]
#         vector = []
        for token in sent:
            vector.append(vocab.lookup_token(token))
        vector.append(self.vocab.end_seq_index)
            
        return vector
    

In [10]:
vocab = Vocabulary(token_to_idx=None, add_unk=True)
nlp = spacy.load(name='en_core_web_sm')

def clean(idx):
    with open(os.path.join(root_dir, str(idx)+'.txt')) as tweet:
        t = tweet.read()
    t = re.sub(r'RT','',t)
    t = re.sub('@[a-zA-z0-9_]+','',t)
    t = re.sub(r'http://[^\s<>"]+|www\.[^\s<>"]+','',t)
    t = re.sub(r'&#[0-9]+','',t)
    t = re.sub(r'[0-9]+','',t)
    t = re.sub(r'[^\w\s]+', '',t)
    t = t.strip()
    t = " ".join(t.split())
    t = [token.lemma_ for token in nlp(t) if token.text not in STOP_WORDS]
    return t

In [11]:
# tqdm.pandas()
# cleaned_tweets = train_data.tweet_id.progress_apply(clean)
text = []

In [12]:
with open('cleaned_tweets.txt', 'r') as  txt:
    cleaned_tweets = txt.readlines()
cleaned_tweets = cleaned_tweets[0].split(" ")
for i in tqdm(range(len(cleaned_tweets))):
    word = cleaned_tweets[i]
    text.append(word)

100%|██████████| 1036166/1036166 [00:00<00:00, 3078118.28it/s]


In [13]:
# with open('cleaned_tweets.txt', 'w') as txt:
#     txt.writelines([" ".join(tweet) for tweet in list(cleaned_tweets)])

In [14]:
# cleaned_tweets = list(cleaned_tweets.copy())
# for i in tqdm(range(len(cleaned_tweets))):
#     for word in cleaned_tweets[i]:
#         text.append(word)

In [15]:
len(text)
from collections import Counter
count_dict = Counter(text).most_common(len(set(text))-5000)
for tup in tqdm(count_dict):
    vocab.add_token(tup[0])

100%|██████████| 158469/158469 [00:00<00:00, 1437354.38it/s]


In [16]:
train_data = Sarcastic_tweets(train_df,nlp, vocab)
test_data = Sarcastic_tweets(test_df, nlp, vocab)

In [17]:
train_data[0]

{'tweet': tensor([     1,   2218,      3,  16399,    126,   9409,  11028,      0,      7,
           3025,   8938,      3,     25,    642,  11456, 113220,     12,     16,
              2]),
 'label': 0}

In [18]:
len(train_df[train_df.sarcastic==0]) / len(train_df), len(test_df[test_df.sarcastic==0]) / len(test_df)

(0.5085446715428259, 0.5085614682661745)

In [19]:
def load_glove(filepath):
    """Loads the glove embeddings
    
    Args:
        filepath (str): path to the glove embeddings file
    Return:
        word_to_index (dict): Mappings from word to index
        embeddings (np.array): Embeddings of the words in the vocabulary
    """
    word_to_index = {}
    embeddings = []
    with open(filepath, 'r', encoding='utf-8') as fp:
        for index, line in enumerate(fp):
            line = line.split(" ")
            word_to_index[line[0]] = index
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    
    return word_to_index, np.array(embeddings)

def make_embedding_matrix(filepath, words):
    """Create embedding matrix for a specific set of words
    Args:
        word_to_index (dict) : mapping of word to index
        embeddings (list): embeddings of words
        words (list): List of words in the dictionary
    Returns:
        final_embeddings (np..array) : embedding matrix
    """
    
    word_to_idx, embeddings = load_glove(filepath)
    embedding_size = embeddings.shape[1]
    final_embeddings = np.zeros((len(words), embedding_size))
    
    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.zeros(embedding_size)
            final_embeddings[i, :] = embedding_i
            
    return final_embeddings

In [21]:
words=[]
for idx in tqdm(range(0, vocab.__len__())):
    words.append(vocab.lookup_index(idx))
embs = make_embedding_matrix('/home/karthiktiwari/Data/glove.6B/glove.6B.50d.txt', words)

100%|██████████| 158472/158472 [00:00<00:00, 1517817.82it/s]


In [22]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, embedding_weights, num_layers = 1,bidirectional = False):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        self.word_embeddings = nn.Embedding.from_pretrained(embedding_weights, freeze=False, padding_idx=0)
        self.lstm = nn.LSTM(input_size = self.embedding_dim, hidden_size = hidden_dim,
                            num_layers = num_layers, bidirectional = bidirectional,batch_first=True)
        self.fc = nn.Linear(hidden_dim, label_size)
 
    def forward(self, sentences, train = True):
        embeds = self.word_embeddings(sentences)
        packed_outputs, _ = self.lstm(embeds)
        outputs = self.fc(packed_outputs[:,-1,:].reshape(self.batch_size,-1))
        return outputs

In [23]:
class BidirectionalLSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, embedding_weights, num_layers = 1,bidirectional = False):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        self.attention = nn.MultiheadAttention(embed_dim=self.embedding_dim, num_heads=1)
        self.word_embeddings = nn.Embedding.from_pretrained(embedding_weights, freeze=False, padding_idx=0)
        self.lstm = nn.LSTM(input_size = self.embedding_dim, hidden_size = hidden_dim,
                            num_layers = num_layers, bidirectional = bidirectional,batch_first=True)
        if bidirectional:
            self.fc = nn.Linear(hidden_dim*2, label_size)
        else:
            self.fc = nn.Linear(hidden_dim, label_size)
 
    def forward(self, sentences, train = True):
        embeds = self.word_embeddings(sentences)
        embeds = embeds.permute(0,1,2)
        attn_output, _ = self.attention(embeds, embeds, embeds)
        packed_outputs, (hidden,cell) = self.lstm(attn_output.permute(0,1,2))
        dense_outputs = self.fc(packed_outputs[:,-1,:])
        outputs = dense_outputs
        return outputs

In [24]:
class AttnBasedLSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size, embedding_weights, num_layers = 1,bidirectional = False):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.embedding_dim = embedding_dim
        self.attention = nn.MultiheadAttention(embed_dim=self.embedding_dim, num_heads=1)
        self.word_embeddings = nn.Embedding.from_pretrained(embedding_weights, freeze=False, padding_idx=0)
        self.lstm = nn.LSTM(input_size = self.embedding_dim, hidden_size = hidden_dim,
                            num_layers = num_layers, bidirectional = bidirectional,batch_first=True)
        if bidirectional:
            self.fc = nn.Linear(hidden_dim*2, label_size)
        else:
            self.fc = nn.Linear(hidden_dim, label_size)
 
    def forward(self, sentences, train = True):
        embeds = self.word_embeddings(sentences)
        embeds = embeds.permute(0,1,2)
        attn_output, _ = self.attention(embeds, embeds, embeds)
        packed_outputs, (hidden,cell) = self.lstm(attn_output.permute(0,1,2))
        dense_outputs = self.fc(packed_outputs[:,-1,:])
        outputs = dense_outputs
        return outputs

In [25]:
BATCH_SIZE = 128
nlabel = 2
num_layers = 2
hidden_dim = 64
EMBEDDING_DIM = embs.shape[1]
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = LSTMClassifier(embedding_dim=EMBEDDING_DIM,hidden_dim=hidden_dim,label_size=nlabel, batch_size=BATCH_SIZE, embedding_weights=torch.from_numpy(embs).float(), num_layers=num_layers, bidirectional = False)
# model = BidirectionalLSTM(embedding_dim=EMBEDDING_DIM,hidden_dim=hidden_dim,label_size=nlabel, batch_size=BATCH_SIZE, embedding_weights=torch.from_numpy(embs).float(), num_layers=num_layers, bidirectional = True)
model = model.to(device)
 
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)
loss_function = nn.CrossEntropyLoss()
 
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [26]:
#Rewrite collate_ FN function, whose input is the sample data of a batch
def collate_fn(batch):
	#Because token_ List is a variable length data, so you need to use a list to load the token of the batch_ list
    token_lists = [item['tweet'] for item in batch]
    #Each label is an int. we take out all the labels in the batch and reassemble them
    labels = [item['label'] for item in batch]
    #Converting labels to tensor
    labels = torch.LongTensor(labels)
    return {
    'token_list': torch.nn.utils.rnn.pad_sequence(token_lists, batch_first=True),
    'label': labels,
    }

#When using dataloader to load data, pay attention to collate_ The FN parameter passes in an overridden function
trainset = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn, drop_last = True)
testset = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn, drop_last = True)

In [27]:
for batch in trainset:
    print(batch['token_list'])
    print(batch['label'])
    break

tensor([[    1,  5635,  1556,  ...,     0,     0,     0],
        [    1,    25,  1297,  ...,     0,     0,     0],
        [    1,     0, 24347,  ...,     0,     0,     0],
        ...,
        [    1, 37793,   192,  ...,     0,     0,     0],
        [    1,   645,  3914,  ...,     0,     0,     0],
        [    1,   334,     6,  ...,     0,     0,     0]])
tensor([0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
        1, 0, 0, 1, 0, 1, 0, 1])


In [28]:
from tqdm import tqdm
epochs=5
for epoch in range(epochs):
    time.sleep(1)
    total_train_loss = 0.0
    total_val_loss = 0.0
    total_train_acc = 0.0
    total_val_acc = 0.0
    model.train()
    for i, batch in enumerate(tqdm(trainset)):
        feature, label = batch['token_list'].to(device), batch['label'].to(device)
#         batch_length = torch.tensor(33, dtype = torch.int64).unsqueeze(0)
        optimizer.zero_grad()
        output =  model(feature).squeeze()
        loss = loss_function(output, label)
        acc=categorical_accuracy(output,label)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        total_train_acc += acc.item() 
    
    model.eval()
    for i, batch in enumerate(tqdm(testset)):
        feature, label = batch['token_list'].to(device), batch['label'].to(device)
#         batch_length = torch.tensor(33, dtype = torch.int64).unsqueeze(0)
        optimizer.zero_grad()
        output =  model(feature).squeeze()
        loss = loss_function(output, label)
        acc=categorical_accuracy(output,label)
        total_val_loss += loss.item()
        total_val_acc += acc.item() 
    scheduler.step()    
    print(f"Train loss on epoch {epoch + 1}={total_train_loss/len(trainset)}")
    print(f"Val loss on epoch {epoch + 1}={total_val_loss / len(testset)}")
    print(f"Train accuracy on epoch {epoch+1} = {total_train_acc/len(trainset)}")
    print(f"Val accuracy on epoch {epoch+1} = {total_val_acc/len(testset)}")

100%|██████████| 76/76 [00:52<00:00,  1.46it/s]


Train loss on epoch 1=0.46030625523659435
Val loss on epoch 1=0.18419986767204186
Train accuracy on epoch 1 = 0.7200159671532846
Val accuracy on epoch 1 = 0.9293791118421053


 37%|███▋      | 28/76 [00:20<00:35,  1.34it/s]

In [ ]:
model_name = input()
torch.save(model.state_dict(), '{}.pth'.format(model_name))

In [ ]:
model.eval()
count = 0
gts = []
model_preds = []
with torch.no_grad():
    for data in tqdm(testset):
        X, y = batch['token_list'].to(device), batch['label'].to(device)

        output = model(X).squeeze()
        acc=categorical_accuracy(output,label)
        model_preds.extend(output.detach().cpu().numpy())
        gts.extend(y.detach().cpu().numpy())
        gt = y.squeeze(dim=1)

        count += (model_preds == gt).sum()

print("Accuracy on Test Data is {0:.4g}%".format(
    (count.item()*100)/len(testset)))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gts, model_preds))

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
auc = metrics.roc_auc_score(gts, model_preds)

false_positive_rate, true_positive_rate, thresolds = metrics.roc_curve(gts, model_preds)

plt.figure(figsize=(10, 8), dpi=100)
plt.axis('scaled')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.title("AUC & ROC Curve")
plt.plot(false_positive_rate, true_positive_rate, 'g')
plt.fill_between(false_positive_rate, true_positive_rate, facecolor='lightgreen', alpha=0.7)
plt.text(0.95, 0.05, 'AUC = %0.4f' % auc, ha='right', fontsize=12, weight='bold', color='blue')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()